<center>

### Instalacije okoline za ARIS laboratorijske vježbe

</center>

ARIS laboratorijske vježbe će se radit na vlastitim računalima koje trebate pripremiti.
Preporuča se korištenje Ubuntu 22.04 LTS operacijskog sustava, na kojem će biti pripremljeni:
- PostgreSQL 14 baza podataka
- docker - za razne servere koji će se pokretati tijekom laboratorija
- minikube - lokalni kubernetes cluster koji će se koristiti za neke od laboratorija
- python 3

Kako bismo smanjili laboratorije koje morate skinuti sa fer web-a, ostavljeni su samo jupyter notebookovi, a ostatak okoline si morate sami posložiti i podesiti.

Laboratorij možete raditi i na macOS i Windows operacijskim sustavima, no u tim slučajevima vam budemo teško mogli asistirati kada stvari neće raditi kako je to zamišljeno. Ukoliko imate takva računala, uvijek možete na njemu napraviti virtualni stroj (recimo pomoću Oracle VirtualBox-a : https://www.virtualbox.org/wiki/Downloads ).

Ubuntu 22.04 LTS možete preuzeti [Ubuntu VirtualBox downloads](https://ubuntu.com/tutorials/how-to-run-ubuntu-desktop-on-a-virtual-machine-using-virtualbox#1-overview).

S obzirom da će se koristiti docker, minikube i da na računalu imate PostgreSQL, minimalna konfiguracija je:
- 4-6 jezgri procesora
- 8-12 GB RAM-a
- 128+ GB diska

Laboratorijske vježbe strukturirane su u niz foldera. U krovnom folderu nalazi se virtualna okolina za python, te osnovne datoteke koje su potrebne za pokretanje *bash* skripti u pojedinim lab folderima, poput:
- *mypassfile* - u koji stavljate password koji se koristi za ```sudo``` naredbu, a treba za administracijske ovlasti u operacijskom sustavu.

Prvo stvaramo virtualnu okolinu koju budemo koristili u svim labovima.

In [ ]:
%%bash
virtualenv ./venv

Zatim instaliramo GNU kompajler i biblioteke potrebne za PostgreSQL razvoj.

In [ ]:
%%bash
sudo -S echo < mypassfile
sudo -S apt install gcc
sudo -S apt install libpq-dev --assume-yes

Na kraju instaliramo python module koji se budu koristili u svim labovima. Python moduli koji će biti specifični za pojedini lab se budu instalirali kao dio jupyter notebooka u tom labu.

Za pytorch uzimamo samo CPU varijantu. Studenti koji imaju NVIDIA grafičke kartice mogu isprobati i punu (CUDA included) verziju tako da maknu ```--extra-index-url``` iz instalacije pytorch modula. CPU only verzija pytorcha je značajnije manja nego puna verzija, te to treba imati na umu kod deploymenta na razne servere.

In [ ]:
%%bash
source ./venv/bin/activate

pip install -U pip
pip install scikit-learn pandas numpy ipykernel matplotlib sqlalchemy seaborn psycopg2 jupyter torchvision tabulate sklearn2pmml imblearn
pip install torch --extra-index-url https://download.pytorch.org/whl/cpu

<center>

### Instalacija postgresql-a. 

</center>

Za 22.04LTS to je verzija 14.
```
sudo apt-get install postgresql
```

Promijenimo password korisnika *postgres* u *postgres* bazi.
```
sudo -u postgres psql
```
Izvršava se SQL naredba kojom se mijenja password *postgres* korisnika
```sql:
ALTER USER postgres PASSWORD 'pgadmin';
EXIT
```

Promijenimo password korisnika *postgres* u operacijskom sustavu u *pgadmin*.
```
sudo passwd postgres
```

Promijenimo konfiguraciju postgresql-a.
```
sudo vi /etc/postgresql/14/main/pg_hba.conf
```
- Promjena auth za korisnika *postgres* iz *peer* u ***md5***
- Dodamo:
    ```
    host all all 172.19.0.0/16 md5
    ```
zatim ide
```
sudo vi /etc/postgresql/14/main/postgresql.conf
```
- Stavljamo * s obzirom da bismo se mogli spajati na bazu iz docker kontejnera
    ```
    listen_addresses='*'
    ```

Restartamo postgresql
```
sudo service postgresql restart
```

Testiramo rad baze podataka i promjene passworda za korisnika *postgres*
```
psql -U postgres
```

Ova baza služi kao backend za mlflow server. Ona se može instalirati i u container varijanti (*docker*). Cijeli proces instalacije mlflow servera na docker se može vidjeti na  
https://towardsdatascience.com/deploy-mlflow-with-docker-compose-8059f16b6039

Prvo stvorimo instancu baze podataka *mlflow_db* koja će poslužiti mlflow serveru za spremanje podataka.  
Environment varijabla ```PGPASSWORD``` nam služi za izbjegavanje prompta u jupyter notebooku, a u realnom životu **izbjegavajte** upisivanje passworda u plain-text obliku!